<a href="https://colab.research.google.com/github/LakshmiMedapati96/DM-Kaggle_Image_Classification/blob/main/Movie_Review_NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries and Loading the dataset**

In [ ]:
# The "encoding='latin-1'" in the below line of code were taken from Reference [1].

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from collections import defaultdict
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv("rt_reviews.csv", encoding='latin-1')
data.head()

**Data Cleaning**

In [ ]:
# All the lines of code in this cell were taken from Reference [3].

# Define function for removing square brackets and special characters
def remove_square_brackets_special_characters(text):
    text = re.sub('\[[^]]*\]','', text)
    text = re.sub(r'[^a-zA-z0-9\s]','',text)
    return text

# Apply function on review column
data['Review'] = data['Review'].apply(remove_square_brackets_special_characters)

# data.head()
data.shape

**Dividing the dataset into Train, Development and Test sets (60:20:20)**

In [ ]:
# All the lines of code in this cell were written by me by referring to Reference [2].

# Split the data into training and remaining datasets
trainset, remset = train_test_split(data, train_size = 0.6)

# Split the remaining dataset into development and test datasets
devset, testset = train_test_split(remset, train_size = 0.5)

trainset.shape, devset.shape, testset.shape

**Build vocabulary as dictionary**

In [ ]:
# All the lines of code in this cell were written by me.

# Defining a function to build vocabulary as dictionary and 
# omitting rare words (if the occurrence is less than five times)
def build_vocab_dict(dataset, flag=False):
    vocab_dict = {}
    for review in dataset["Review"]:
        review = review.split(" ")
        # print(review)
        for word in review:
            word = word.lower()
            if word in vocab_dict:
                vocab_dict[word] += 1
            else:
                vocab_dict[word] = 1
    vocab_dict.pop('')                                     # for removing '' in the dictionary
    if flag:
        return vocab_dict
    total_vocab_dict = {k: v for k, v in vocab_dict.items() if v >= 5}
#print(str(total_word_dict)[:1000] + "...........")
    return total_vocab_dict

**Total words dictionary - Entire dataset and Training data**

In [ ]:
# All the lines of code in this cell were written by me.

total_word_dict = build_vocab_dict(data)                        # dictionary for entire dataset

total_word_dict_train = build_vocab_dict(trainset)              # dictionary for training dataset
print(f"Total words in the dictionary (train set): {total_word_dict_train}")

**Calculating prior probabilities**

In [ ]:
# All the lines of code in this cell were written by me.

# Creating separate dataframes for fresh and rotten datasets
fresh_df = trainset[trainset["Freshness"] == 'fresh']
rotten_df = trainset[trainset["Freshness"] == 'rotten']
#print(fresh_df)
#print(rotten_df)

# Probabilities for fresh and rotten classes
prob_fresh = fresh_df.shape[0]  / (fresh_df.shape[0] + rotten_df.shape[0])
prob_rotten = rotten_df.shape[0]  / (fresh_df.shape[0] + rotten_df.shape[0])

print(f"P(rotten) = {str(prob_rotten)}")
print(f"P(fresh) = {str(prob_fresh)}")

**Creating separate word dictionaries for fresh and rotten classes**

In [ ]:
# All the lines of code in this cell were written by me.

# fresh word dictionary
fresh_word_dict_train = build_vocab_dict(fresh_df)
print(f"Words dictionary for fresh label: {fresh_word_dict_train}")

# rotten word dictionary
rotten_word_dict_train = build_vocab_dict(rotten_df)
print(f"Words dictionary for rotten label: {rotten_word_dict_train}")

**Calculating probability of the occurrence**

In [ ]:
# All the lines of code in this cell were written by me.

# Calculating probability of all the words
total_num_of_words_train = sum(list(total_word_dict_train.values()))

for key, value in total_word_dict_train.items():
    total_word_dict_train[key] = (value, value*100/total_num_of_words_train)
print(f"Probability of the occurence: {str(total_word_dict_train)}")

**Calculating conditional probability based on the sentiment**

In [ ]:
# All the lines of code in this cell were written by me.

# Calculating probability of each word in fresh.
fresh_total_num_of_words = sum(list(fresh_word_dict_train.values()))
fresh_word_prob_dict_train = {}
for key, value in fresh_word_dict_train.items():
    fresh_word_prob_dict_train[key] = value*100/fresh_total_num_of_words
print(f"Conditional probability based on 'fresh' label: {str(fresh_word_prob_dict_train)}")

# Calculating probability of each word in rotten.
rotten_total_num_of_words = sum(list(rotten_word_dict_train.values()))
rotten_word_prob_dict_train = {}
for key, value in rotten_word_dict_train.items():
    rotten_word_prob_dict_train[key] = value*100/rotten_total_num_of_words
print(f"Conditional probability based on 'rotten' label: {str(rotten_word_prob_dict_train)}")

**Calculating accuracy on Development dataset**

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

# Defining a function to calculate accuracy score
def accuracyscore(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == "rotten":
            value = 0
        else:
            value = 1
        if value == predicted[i]:
            correct = correct + 1
            
        # print(actual[i] , predicted[i])
    #print(f"correct = {correct}")
    #print(str(rotten_word_prob_dict_train)[:200])
    return correct * 100.0/ float(len(actual))

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

# Defining a function to predict the class label
def predict(preddata):
    pred = []
    for review in preddata["Review"]:
        vocab_dict = {}
        review = review.split(" ")
        # print(review)
        for word in review:
            word = word.lower()
            if word in vocab_dict:
                vocab_dict[word] += 1
            else:
                vocab_dict[word] = 1
        vocab_dict.pop('', None)                                     # for removing '' in the dictionary
        likelihood_rotten = 1.0
        likelihood_fresh = 1.0
        
        for word in vocab_dict:
            if not word in rotten_word_prob_dict_train:
                rotten_word_prob_dict_train[word] = 0.0

            if not word in fresh_word_prob_dict_train:
                fresh_word_prob_dict_train[word] = 0.0

            likelihood_rotten = likelihood_rotten * rotten_word_prob_dict_train[word]
            likelihood_fresh = likelihood_fresh * fresh_word_prob_dict_train[word]
        
        posterior_rotten = prob_rotten * likelihood_rotten
        posterior_fresh = prob_fresh * likelihood_fresh

        if posterior_rotten > posterior_fresh:
            y_pred = 0
        else:
            y_pred = 1

        pred.append(y_pred)
    return pred

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

dev_pred = predict(devset)

accuracy = accuracyscore(devset['Freshness'].to_list(),dev_pred)
print(f"Accuracy on development set before smoothing : {accuracy}%")

**Applying laplace smoothing**

In [ ]:
# All the lines of code in this cell were written by referring to Reference [4].

# Defining afunction to apply smoothing technique
def laplace_smoothing(word_freq_0, word_freq_1, alpha, vocab_dict):
    word_freq_0_probs = {}
    word_freq_1_probs = {}

    vocab_len = len(vocab_dict)
    for word in vocab_dict:
        word_freq_0_probs[word] = (word_freq_0.get(word, 0) + 1) / (len(word_freq_0) + vocab_len * alpha)
        word_freq_1_probs[word] = (word_freq_1.get(word, 0) + 1) / (len(word_freq_1) + vocab_len * alpha)

    return word_freq_0_probs, word_freq_1_probs

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

# Defining a function to predict the class label with laplace smoothing
def predict_alpha(preddata, alpha, rotten_word_prob_dict_train, fresh_word_prob_dict_train):
    import time
    #print("Alpha prediction started.")
    pred = []
    i = 0
    #print("Iter length : ", len(preddata))
    for review in preddata["Review"]:
        # s_t = time.time()
        i = i+1
        #print(f"Iter value: {i}")
        vocab_dict = {}
        review = review.split(" ")
        # print(review)
        # Create a defaultdict with int as the default factory
        vocab_dict = defaultdict(int)
        import time
        # s_t_2 = time.time()
        for word in review:
            word = word.lower()
            vocab_dict[word] += 1
        vocab_dict.pop('', None)                                    # for removing '' in the dictionary
        # a1_t = time.time()- s_t_2
        # print(f"time taken for word_review: {a1_t}sec")
        likelihood_rotten = 1.0
        likelihood_fresh = 1.0
        # s_t_3 = time.time()
        for word in vocab_dict:
            if not word in rotten_word_prob_dict_train:
                rotten_word_prob_dict_train[word] = 0.0

            if not word in fresh_word_prob_dict_train:
                fresh_word_prob_dict_train[word] = 0.0
            modified_rotten_word_prob_dict_train, modified_fresh_word_prob_dict_train = laplace_smoothing(rotten_word_prob_dict_train, fresh_word_prob_dict_train, alpha, vocab_dict)
            likelihood_rotten = likelihood_rotten * modified_rotten_word_prob_dict_train[word]
            likelihood_fresh = likelihood_fresh * modified_fresh_word_prob_dict_train[word]
        # a_t = time.time()- s_t_3
        # print(f"total time taken for word_vocabdict: {a_t}sec")
        posterior_rotten = prob_rotten * likelihood_rotten
        posterior_fresh = prob_fresh * likelihood_fresh

        if posterior_rotten > posterior_fresh:
            y_pred = 0
        else:
            y_pred = 1

        pred.append(y_pred)
        # print("appended.")
        # a_t = time.time()- s_t
        # print(f"total time taken for iter: {a_t}sec")
    return pred

**Determining the best value for alpha**

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
lap_acc = []
# print(str(rotten_word_prob_dict_train)[:200])
for alpha in alpha_values:
  # print("alpha value:", alpha)
  alpha_preds = predict_alpha(devset, alpha, rotten_word_prob_dict_train, fresh_word_prob_dict_train)
  # print("Alpha prediction ended.")
  alpha_accuracy = accuracyscore (devset["Freshness"].to_list(), alpha_preds)
  print(f'alpha = {alpha}, Accuracy = {alpha_accuracy} %')
  lap_acc.append(alpha_accuracy)

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

plt.plot(alpha_values, lap_acc)
plt.xlabel('alpha')
plt.ylabel('Accuracy')
plt.show()

**Deriving the Top 10 words that predicts each class**

In [ ]:
# All the lines of code in this cell were written by me.

# Top 10 words - fresh
sorted_fresh_train = sorted(fresh_word_dict_train.items(), key=lambda x: x[1], reverse=True)
print(f"Top 10 words that predicts 'fresh' class : {dict(sorted_fresh_train[:10])}")

# Top 10 words - rotten
sorted_rotten_train = sorted(rotten_word_dict_train.items(), key=lambda x: x[1], reverse=True)
print(f"Top 10 words that predicts 'rotten' class : {dict(sorted_rotten_train[:10])}")

**Calculating final accuracy on test dataset with optimal hyperparameter (alpha)**

In [ ]:
# All the lines of code in this cell were taken from Reference [4].

alpha = 10000
test_pred = predict_alpha(testset, alpha, rotten_word_prob_dict_train, fresh_word_prob_dict_train)

lap_test_accuracy = accuracyscore(testset["Freshness"].to_list(), test_pred)
print(f'Final Accuracy on Test Dataset after applying Laplace smoothing : {lap_test_accuracy} %')